# Track 3: Healthcare ML - Day 1
## Objective: Load CMS Data and Explore Columns

In [1]:
import pandas as pd
df = pd.read_csv('C:/Users/shazi/Healthcare_Project/output/cleaned_cms_hospital_info.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5396 entries, 0 to 5395
Data columns (total 38 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   facility_id                                       5396 non-null   object 
 1   facility_name                                     5396 non-null   object 
 2   address                                           5396 non-null   object 
 3   city/town                                         5396 non-null   object 
 4   state                                             5396 non-null   object 
 5   zip_code                                          5396 non-null   int64  
 6   county/parish                                     5396 non-null   object 
 7   telephone_number                                  5396 non-null   object 
 8   hospital_type                                     5396 non-null   object 
 9   hospital_ownership 

,facility_id,facility_name,address,city/town,state,zip_code,county/parish,telephone_number,hospital_type,hospital_ownership,...,count_of_readm_measures_better,count_of_readm_measures_no_different,count_of_readm_measures_worse,readm_group_footnote,pt_exp_group_measure_count,count_of_facility_pt_exp_measures,pt_exp_group_footnote,te_group_measure_count,count_of_facility_te_measures,te_group_footnote
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,Acute Care Hospitals,Government - Hospital District or Authority,...,1,8,2,NaN,8,8,NaN,12,10,NaN
1,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,Acute Care Hospitals,Government - Hospital District or Authority,...,0,8,1,NaN,8,8,NaN,12,12,NaN
2,010006,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,FLORENCE,AL,35630,LAUDERDALE,(256) 629-1000,Acute Care Hospitals,Proprietary,...,0,7,2,NaN,8,8,NaN,12,11,NaN
3,010007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,(334) 493-3541,Acute Care Hospitals,Voluntary non-profit - Private,...,0,7,0,NaN,8,8,NaN,12,7,NaN
4,010008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW,(334) 335-3374,Acute Care Hospitals,Proprietary,...,0,2,0,NaN,8,Not Available,5.0,12,6,NaN


In [2]:
#Check for nulls:
df.isnull().sum().sort_values(ascending=False)


te_group_footnote                                   4509
readm_group_footnote                                4315
mort_group_footnote                                 3527
safety_group_footnote                               3406
pt_exp_group_footnote                               3207
meets_criteria_for_birthing_friendly_designation    3163
hospital_overall_rating_footnote                    2777
state                                                  0
zip_code                                               0
address                                                0
facility_name                                          0
facility_id                                            0
city/town                                              0
county/parish                                          0
hospital_overall_rating                                0
mort_group_measure_count                               0
count_of_mort_measures_better                          0
count_of_facility_mort_measures

In [10]:
#Drop columns that are mostly null or irrelevant
#df = df.drop(columns=['hospital_overall_rating_footnote', 'mort_group_footnote'])


In [8]:
#Define Target Column: Let’s create a binary column:
df['count_of_readm_measures_worse'] = pd.to_numeric(
    df['count_of_readm_measures_worse'], errors='coerce'
)


In [11]:
df['readmission_flag'] = df['count_of_readm_measures_worse'].apply(lambda x: 1 if x > 0 else 0)


In [12]:
df = df.dropna(subset=['count_of_readm_measures_worse'])


In [13]:
df.to_csv("C:/Users/shazi/Healthcare_Project/data/cleaned_cms_ml.csv", index=False)
